In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False
import cv2 

In [3]:
def load_frames(source):
    arr = None
    with open(source, 'rb') as f:
        arr = np.load(f) 
    return arr    
with open('../speed_per_20.npz', 'rb') as f:
    arr = np.load(f)

In [4]:

arr.shape 

(8, 540, 960, 2)

In [5]:
total_frame = arr.shape[0]
total_observe_line = arr.shape[1]
total_flow_direction_line = arr.shape[2]

In [6]:
def frames_to_bgrs(arr):
    total_observe_line = arr.shape[1]
    total_flow_direction_line = arr.shape[2]
    fx, fy = arr[:, :,:,0], arr[:, :,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    
    # create hsv figure for flow vecoity
    
    hsv = np.zeros((ang.shape[0], total_observe_line, total_flow_direction_line, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    
    # hsv to bgr
    bgr = np.array([cv2.cvtColor(hsv[i], cv2.COLOR_HSV2RGB).tolist() for i in range(8)])
    return bgr

In [7]:
bgr = frames_to_bgrs(arr)

In [8]:
bgr.shape 

(8, 540, 960, 3)

In [14]:
# input:
#   bgr shape [frame, w, h, 3] 
#   frame_index : number
#   y_ind : number 
#   step : number (measure intetrval)
#   uninterval_index : None | iterable (make efferts only when step is not a const)
# return:
#   
def calc_flow_index(bgr, frame_index = 0, y_ind = 120, step = 60, uninterval_indicies = None, surface_width = 420):
    different_map = np.where(bgr[frame_index].sum(axis = 2) == 0, 0, 255 )
    start_index = different_map[y_ind, :].tolist().index(255)
    end_index = bgr.shape[2] - different_map[y_ind, :].tolist()[::-1].index(255) - 1
    if not uninterval_indicies:
        indices = np.array([*range(0, surface_width + 1, step)])
    else:
        indices = np.array(uninterval_indicies)
    map_indices = (indices * (end_index - start_index) / surface_width).astype("int")
    return map_indices

In [15]:
calc_flow_index(bgr, frame_index=1)

array([  0,  78, 157, 236, 315, 394, 473, 552])

In [22]:
def calc_frame_flow_index(frame, y_ind = 120, step = 60, uninterval_indicies = None, surface_width = 420):
    different_map = np.where(frame.sum(axis = 2) == 0, 0, 255 )
    start_index = different_map[y_ind, :].tolist().index(255)
    end_index = frame.shape[1] - different_map[y_ind, :].tolist()[::-1].index(255) - 1
    if not uninterval_indicies:
        indices = np.array([*range(0, surface_width + 1, step)])
    else:
        indices = np.array(uninterval_indicies)
    map_indices = (indices * (end_index - start_index) / surface_width).astype("int")
    return map_indices

In [23]:
calc_frame_flow_index(bgr[1])

array([  0,  78, 157, 236, 315, 394, 473, 552])